# 7-Slot Nerdle Solver Test
We prove (by brute-force) that you can always solve mini-Nerdle in at most $4$ guesses regardless of the starting expression, provided you use the optimal strategy. The worst start having repeating numbers and thus less information, e.g. `10-5=5`. The best start has all different numbers: `28/7=4`, which needs at most $3$ guesses and $2.65 \pm 0.5$ guesses.

In [2]:
%load_ext autoreload
%autoreload 2

import collections
import itertools
import numpy as np
import matplotlib.pyplot as plt

import nerdle
import score as s
import generator
from nerdle import Hint, NerdleData

In [41]:
# Mini-Nerdle.
NUM_SLOTS = 7
SCORE_DB_FILE = "nerdle{}.db".format(NUM_SLOTS) 

solver_data = nerdle.create_solver_data(NUM_SLOTS, SCORE_DB_FILE)

0 / 6661 (0.0%) completed
333 / 6661 (5.0%) completed
666 / 6661 (10.0%) completed
999 / 6661 (15.0%) completed
1332 / 6661 (20.0%) completed
1665 / 6661 (25.0%) completed
1998 / 6661 (30.0%) completed
2331 / 6661 (35.0%) completed
2664 / 6661 (40.0%) completed
2997 / 6661 (45.0%) completed
3330 / 6661 (50.0%) completed
3663 / 6661 (55.0%) completed
3996 / 6661 (60.0%) completed
4329 / 6661 (65.0%) completed
4662 / 6661 (70.0%) completed
4995 / 6661 (75.0%) completed
5328 / 6661 (80.0%) completed
5661 / 6661 (85.0%) completed
5994 / 6661 (90.0%) completed
6327 / 6661 (95.0%) completed
6660 / 6661 (100.0%) completed


In [23]:
d = solver_data.score_db
print(len(d), len(solver_data.answers))
for key in list(d.keys())[:10]:
    print(key, "".join(map(str, key[0])) + "=" + str(key[1]), len(d[key]))

6661 6661
98-4=94 9=8 6661
7-8+7=6 7=- 6661
75-3=72 7=5 6661
8-2*1=6 8=- 6661
51+5=56 5=1 6661
83-2=81 8=3 6661
23-20=3 2=3 6661
68/1=68 6=8 6661
9+5-9=5 9=+ 6661
94/47=2 9=4 6661


## Example Usage

In [48]:
scorer = s.Scorer(NUM_SLOTS)
score = scorer("1*1*6=6", "99/9=11")
print(score, 
      s.score_to_hint_string(score, NUM_SLOTS),
      s.score_to_hints(score, NUM_SLOTS) == [Hint.MISPLACED, Hint.INCORRECT, Hint.MISPLACED, Hint.INCORRECT, Hint.INCORRECT, Hint.MISPLACED, Hint.INCORRECT]
     )

2082 ?-?--?- True


In [25]:
# A good initial guess significantly reduces the number of answers. In this case, from
# 206 to 10.
guess_history, hint_history, answer_size_history =  nerdle.NerdleSolver(solver_data).solve("99/9=11", initial_guess= "1+6-7=0", debug=True)

--> guess 1+6-7=0
score ?----?- 2050 answers 128
--> guess 2*19=38
score --?++-- 352 answers 1
--> guess 99/9=11
score +++++++ 5461 answers 1


## Benchmark
This is a fast in-memory dict implementation.

In [26]:
%time guess_history, hint_history, answer_size_history = nerdle.NerdleSolver(solver_data).solve("99/9=11", initial_guess="1+6-7=0")

CPU times: user 4.96 s, sys: 14.4 ms, total: 4.97 s
Wall time: 4.98 s


In [ ]:
answer = "99/9=11"
solver = nerdle.NerdleSolver(solver_data)
for start in list(solver_data.answers)[:5]:
    print(start)
    %time solver.solve(answer, initial_guess=start) 

## Initial Guess Optimization
Assuming something with a lot of different numbers and operators is best.

In [ ]:
# answer = "99/9=11"
# solutions = [nerdle.NerdleSolver(solver_data).solve(answer, initial_guess=start) 
#              for start in solver_data.answers]
# n = np.array([len(solution[0]) for solution in solutions])
# num_answers = len(solver_data.answers)
# compression_ratio = num_answers / np.array([solution[2][0] for solution in solutions])
# print(np.mean(compression_ratio), np.std(compression_ratio))

In [17]:
start = "1+6-7=0"
solutions = [nerdle.NerdleSolver(solver_data).solve(answer, initial_guess=start) 
             for answer in solver_data.answers]               
n = np.array([len(solution[0]) for solution in solutions])
num_answers = len(solver_data.answers)
compression_ratio = num_answers / np.array([solution[2][0] for solution in solutions])
print(np.mean(compression_ratio), np.std(compression_ratio))

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

ax = axs[0]
ax.hist(n);
ax.set_title("#Guesses for start {}".format(start));

ax = axs[1]
ax.hist(compression_ratio);
ax.set_title("Compression Ratio Distribution, start {}".format(start));

KeyboardInterrupt: 

## Profiling

In [21]:
import cProfile
import pstats
from pstats import SortKey

AttributeError: '_NerdleDataDict' object has no attribute 'score_db'

In [42]:
cProfile.run('guess_history, hint_history, answer_size_history = nerdle.NerdleSolver(solver_data).solve("99/9=11", initial_guess="1+6-7=0", debug=True)', 'stats')
p = pstats.Stats('stats')
p.sort_stats(SortKey.CUMULATIVE).print_stats(20)

--> guess 1+6-7=0
score ?----?- 2050 answers 128
--> guess 2*19=38
score --?++-- 352 answers 1
--> guess 99/9=11
score +++++++ 5461 answers 1
Wed Sep 14 09:28:01 2022    stats

         1039396 function calls (1039375 primitive calls) in 7.575 seconds

   Ordered by: cumulative time
   List reduced from 60 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    7.575    7.575 {built-in method builtins.exec}
        1    0.001    0.001    7.575    7.575 <string>:1(<module>)
        1    0.000    0.000    7.572    7.572 /Users/olivne/oren/nerdle-solver/nerdle.py:146(solve)
        1    0.000    0.000    7.572    7.572 /Users/olivne/oren/nerdle-solver/nerdle.py:152(solve_adversary)
        3    0.017    0.006    7.571    2.524 /Users/olivne/oren/nerdle-solver/nerdle.py:181(make_guess)
        3    0.000    0.000    7.370    2.457 /Users/olivne/oren/nerdle-solver/nerdle.py:91(restrict_by_answers)
        3    0.3

In [47]:
cProfile.run('solver_data = nerdle.create_solver_data(7, "nerdle7_test.db", overwrite=True, max_answers=1000)', 'stats')
p = pstats.Stats('stats')
p.sort_stats(SortKey.CUMULATIVE).print_stats(20)

0 / 1000 (0.0%) completed
50 / 1000 (5.0%) completed
100 / 1000 (10.0%) completed
150 / 1000 (15.0%) completed
200 / 1000 (20.0%) completed
250 / 1000 (25.0%) completed
300 / 1000 (30.0%) completed
350 / 1000 (35.0%) completed
400 / 1000 (40.0%) completed
450 / 1000 (45.0%) completed
500 / 1000 (50.0%) completed
550 / 1000 (55.0%) completed
600 / 1000 (60.0%) completed
650 / 1000 (65.0%) completed
700 / 1000 (70.0%) completed
750 / 1000 (75.0%) completed
800 / 1000 (80.0%) completed
850 / 1000 (85.0%) completed
900 / 1000 (90.0%) completed
950 / 1000 (95.0%) completed
Wed Sep 14 09:29:42 2022    stats

         2830597 function calls in 4.903 seconds

   Ordered by: cumulative time
   List reduced from 46 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.941    4.941 {built-in method builtins.exec}
        1    0.000    0.000    4.923    4.923 /Users/olivne/oren/nerdle-solver/nerdle.py:207(create_solv

In [59]:
import score_guess as sg
cProfile.run('for _ in range(200000): sg.score_guess("1*1*6=6", "99/9=11")', 'stats')
p = pstats.Stats('stats')
p.sort_stats(SortKey.CUMULATIVE).print_stats(20)

Wed Sep 14 09:55:45 2022    stats

         200003 function calls in 0.535 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.535    0.535 {built-in method builtins.exec}
        1    0.041    0.041    0.534    0.534 <string>:1(<module>)
   200000    0.493    0.000    0.493    0.000 {score_guess.score_guess}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


